# Acompanhamento - Histórico Jira

## GRUPO SEI

---

### Bibliotecas

In [9]:
from datetime import date, datetime, timedelta
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import numpy as np
import requests
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots

### Leituras dos CSVs

In [104]:
# --------------------------------------------------
# DATAS
# --------------------------------------------------
sprints = {
    #"01":{"startday":"10-01-2023", "endday":"27-01-2023"},
    #"02":{"startday":"30-01-2023", "endday":"17-02-2023"},
    "03":{"startday":"27-02-2023", "endday":"10-03-2023"},
    "04":{"startday":"13-03-2023", "endday":"31-03-2023"},
    "05":{"startday":"03-04-2023", "endday":"19-04-2023"},
    "06":{"startday":"24-04-2023", "endday":"12-05-2023"},
    "07":{"startday":"15-05-2023", "endday":"02-06-2023"},
    #"08":{"startday":"05-06-2023", "endday":"23-06-2023"},
    #"09":{"startday":"--2023", "endday":"--2023"},
    #"10":{"startday":"--2023", "endday":"--2023"},
    #"11":{"startday":"--2023", "endday":"--2023"},
    #"12":{"startday":"--2023", "endday":"--2023"},
    #"13":{"startday":"--2023", "endday":"--2023"},
    #"14":{"startday":"--2023", "endday":"--2023"},
    #"15":{"startday":"--2023", "endday":"--2023"},
    #"16":{"startday":"--2023", "endday":"--2023"},
    #"17":{"startday":"--2023", "endday":"--2023"},
}


HOJE = date.today().strftime("%d-%m-%Y")   
today = datetime.strptime(HOJE, "%d-%m-%Y")


# --------------------------------------------------
# Leitura do CSV
# --------------------------------------------------
atividade = {}

for sprint in sprints.keys():
    for day in ["startday", "endday"]:
        if datetime.strptime(sprints[sprint][day], "%d-%m-%Y") < today:
            linhas = open("Sprint{}/csv/Sprint_{}_{}.csv".format(sprint, sprint, sprints[sprint][day]),'r').read()

            fields = (linhas.split("Status Category\n")[0]+"Status Category").split(',')

            linhas = linhas.split("Status Category\n")[1]
            linhas = linhas.replace("Done\n","Done\n\r\r\r").replace("In Progress\n","In Progress\n\r\r\r").replace("To Do\n","To Do\n\r\r\r")
            linhas = linhas.replace(", "," ")
            linhas = linhas.replace(",height",";")
            linhas = linhas.replace("1,2 m","1.2 m")
            linhas = linhas.replace("52,5","52.5")

            if sprint not in atividade.keys():
                atividade[sprint] = {}

            atividade[sprint][day] = linhas.split("\n\r\r\r")[:-1]
            atividade[sprint]["fields{}".format(day)] = fields
            
# --------------------------------------------------



### Tratamento dos dados e informações a coletar

In [105]:
# --------------------------------------------------
# Variáveis a analisar
# --------------------------------------------------
sprintStatus = {}

status = {
            "Done":{"pontos":0, "natividades":0},
            "In Progress":{"pontos":0, "natividades":0},
            "To Do":{"pontos":0, "natividades":0},
            "First Day":{"pontos":0, "natividades":0},
         }



# --------------------------------------------------
# TRATAMENTO DOS DADOS - POR ATIVIDADE (Cada atividade é um item da lista)
# --------------------------------------------------
for currentSprint in sprints.keys():
    for day in ["startday", "endday"]:
        if datetime.strptime(sprints[currentSprint][day], "%d-%m-%Y") < today:
            if currentSprint not in sprintStatus.keys():
                sprintStatus[currentSprint] = {
                    "Done":{"pontos":0, "natividades":0},
                    "In Progress":{"pontos":0, "natividades":0},
                    "To Do":{"pontos":0, "natividades":0},
                    "Last Day":{"pontos":0, "natividades":0},
                    "First Day":{"pontos":0, "natividades":0},
                }

            activities = atividade[currentSprint][day]

            for i in range(len(activities)):
    #            print((len(activities[i].split(","))))
                activityPoint = float(activities[i].split(",")[atividade[currentSprint]["fields{}".format(day)].index("Custom field (Story Points)")])
                activityStatus = activities[i].split(",")[atividade[currentSprint]["fields{}".format(day)].index("Status Category")]
                activityResolved = activities[i].split(",")[atividade[currentSprint]["fields{}".format(day)].index("Resolved")]


                if day == "startday":
                    sprintStatus[currentSprint]["First Day"]["pontos"] += activityPoint
                    sprintStatus[currentSprint]["First Day"]["natividades"] += 1

                else:    
                    for stat in ["Done", "In Progress", "To Do"]:
                        if (stat in activityStatus):
                            sprintStatus[currentSprint][stat]['pontos'] += activityPoint
                            sprintStatus[currentSprint]["Last Day"]["pontos"] += activityPoint
                            sprintStatus[currentSprint][stat]['natividades'] += 1
                            sprintStatus[currentSprint]["Last Day"]["natividades"] += 1

# --------------------------------------------------

### Plots

In [101]:
# --------------------------------------------------
# PLOTS
# --------------------------------------------------
plot_data = {"sprints":[], "plannedActivities":[], "plannedPoints":[], "finalActivities":[], "finalPoints":[], "doneActivities":[], "donePoints":[]}
for sprint in sprints.keys():
    plot_data["sprints"].append(sprint)
    plot_data["plannedActivities"].append(int(sprintStatus[sprint]["First Day"]["natividades"]))
    plot_data["plannedPoints"].append(int(sprintStatus[sprint]["First Day"]["pontos"]))
    plot_data["doneActivities"].append(int(sprintStatus[sprint]["Done"]["natividades"]))
    plot_data["donePoints"].append(int(sprintStatus[sprint]["Done"]["pontos"]))
    plot_data["finalActivities"].append(int(sprintStatus[sprint]["Last Day"]["natividades"]))
    plot_data["finalPoints"].append(int(sprintStatus[sprint]["Last Day"]["pontos"]))



months = plot_data["sprints"]

fig = go.Figure()
fig.add_trace(go.Bar(
    x=months,
    y=plot_data["plannedActivities"],
    name='1° dia',
    marker_color='indianred'
))
fig.add_trace(go.Bar(
    x=months,
    y=plot_data["finalActivities"],
    name='Planejados (ultimo dia)',
    #marker_color='indianred',
))
fig.add_trace(go.Bar(
    x=months,
    y=plot_data["doneActivities"],
    name='Realizados (ultimo dia)',
    marker_color='lightsalmon'
))


# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='group', xaxis_tickangle=-45)
fig.update_layout(title_text='# Atividades Planejadas x Realizadas - Backlog do sprint')
fig.show()

In [102]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=months,
    y=plot_data["plannedPoints"],
    name='Planejados (1° dia)',
    marker_color='indianred',
))
fig.add_trace(go.Bar(
    x=months,
    y=plot_data["finalPoints"],
    name='Planejados (ultimo dia)',
    #marker_color='indianred',
))
fig.add_trace(go.Bar(
    x=months,
    y=plot_data["donePoints"],
    name='Realizados (ultimo dia)',
    marker_color='lightsalmon'
))

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='group', xaxis_tickangle=-45)
fig.update_layout(title_text='# Pontos Planejados x Realizados')
fig.show()